# Cleaning Data (Part 1)
The purpose of this notebook is to read in raw excel data for multiple years, rename and trim columns, append cleaned files into a single dataframe and export this dataframe as an excel file.

# Table of Contents
1. [Setting up Python](#Setting-up-Python)
    
    1. [Setting the Location](#Setting-the-Location)
    
    2. [Importing Necessary Packages](#Importing-Necessary-Packages)
    
    3. [Functions](#Functions)
    
    4. [Preparing for a Save](#Preparing-for-a-Save)  

2. [Handling Columns](#Handling-Columns)
    
    1. [Find Unique Column Names](#Find-Unique-Column-Names)
    
    2. [Eliminate Unnecessary Columns](#Eliminate-Unnecessary-Columns)
    
    3. [Combine Synonyms](#Combine-Synonyms)

3. [Reading and Appending Data](#Reading-and-Appending-Data)

4. [Exporting Data](#Exporting-Data)
5. [Where to Next](#Where-to-Next)

# Setting up Python
[Top](#TOC)

[Setting the Location](#SettingLoc)
    
[Importing Necessary Packages](#ImportingPackages)
    
[Getting Data](#GettingData)
    
[Preparing for a Save](#PreparingSave)

## Importing Necessary Packages

[Top](#TOC)

[Setting Up Python](#SettingUp)

Here we import necessary packages. 
This chunk may take a while.

In [1]:
import pandas as pd
# import numpy as np
import glob,os

# increase print limit
pd.options.display.max_rows = 99999
pd.options.display.max_colwidth = 50
pd.set_option('mode.sim_interactive', True)

## Functions
[Back to: Top](#TOC)

[Back to: Setting Up Python](#SettingUp)

1. [xlcolshape](#xlcolshape)

2. [xluniquecol2](#xluniquecol2)

3. [colmatchtodict](#colmatchtodict)

4. [findsyn](#findsyn)

5. [readnsplit](#readnsplit)

6. [mapndrop](#mapndrop)

7. [namefile](#namefile)

In [2]:
def xlcolshape(file, verbose = True):
    """xlcolshape takes a file name as a string and returns the shape of the excel file"""
    assert isinstance(verbose,bool),"'verbose' must be bool not,{}".format(type(verbose))
    dictionary = {}
    for sheet in pd.ExcelFile(file).sheet_names:
        try:
            tmp = pd.read_excel(file,sheet_name =sheet).shape
            dictentry = file+'_'+sheet
            dictionary[dictentry] = tmp
            if verbose == True:
                print("Doing stuff you asked me to do for file \'{}\',sheet \'{}\' programmer person."\
                      .format(file, sheet))            
        except:
            print("This didn't work for file {}, sheet {}".format(file, sheet))
            
    return dictionary

[Back to: Functions](#Functions)

In [3]:
def xluniquecol2(file, header = 0, verbose=True):
    tmp = []
    for sheet in pd.ExcelFile(file).sheet_names:
        if (('species' in pd.read_excel(file,sheet_name=sheet, header = header).columns)\
            or('Species' in pd.read_excel(file,sheet_name=sheet, header = header).columns)):
            try:
                tmp = list(set(tmp+list(pd.read_excel(file,sheet_name=sheet).columns)))
                if verbose==True:
                    print("Doing stuff you asked me to do for file \'{}\',sheet \'{}\' programmer person."\
                          .format(file,sheet))
                res = tmp
            except:
                print("This didn't work for file {}, sheet {}".format(file,sheet))
        else:
            print("Check columns for file {}.".format(file))
            res = None
    return res
            

[Back to: Functions](#Functions)

In [4]:
def colmatchtodict(x,series, dictsource, key= None):
    """This takes a string, x, and a looks for values in a series that match that contain that string.
    Those values which match are returned as values in a python dict for the key, key.""" 
    assert isinstance(series,pd.Series)
    if key is None:
        key = x
    tmp = series[series.astype(str).str.contains(x,case = False)].tolist()
    dictsource[key] = tmp
    return dictsource
    

[Back to: Functions](#Functions)

In [5]:
def findsyn (name,dictionary, verbose = True):
    """
    *findsyn* checks searches the values of the dict *dictionary* for the string, *name* and returns 
    the key for the key,value pair to which *name* belongs.
    """
    tmp = pd.DataFrame({'preferredcol':list(dictionary.keys()),'synonymns':list(dictionary.values())})
    try:
        res = list(tmp.preferredcol[tmp.synonymns.apply(lambda x:name in x)])[0]
    except:
        res = None
        if verbose == True:
            print("No value matching \"{}\" was found in the dictionary.".format(name))
    return res


[Back to: Functions](#Functions)

In [6]:
def readnsplit(file,newsourcefolder,dtype=None,verbose=True):
    """
    This function reads an excel file, splits its sheets into separate files and saves them to folder
    *newsourcefolder*.
    """
    suffix = '.'+file.split('.')[1]
    prefix = file[:-len(suffix)]
    for sheet in pd.ExcelFile(file).sheet_names:
        try:
            splitfile = newsourcefolder+'/'+prefix+'_'+sheet+suffix
            tmp = pd.read_excel(file,dtype=dtype, sheet_name=sheet).to_excel(splitfile,index=False)
            if verbose==True:
                print("Success!  \'{}\',sheet \'{}\' has been saved to {} and the corresponding\
                google drive file as {}.".format(file,sheet,newsourcefolder,splitfile))
            continue
        except:
            print("Unable to save \'{}\',sheet \'{}\' as a separate file.".format(file,sheet))         


[Back to: Functions](#Functions)

In [7]:
def mapndrop(df,dictionary,verbose=True):
    """
    This function renames columns in *df* deemed synonymous according to a dict,
    *dictionary*, and drops unnecessary columns before returning the cleaner dataframe.
    """
    try:
        df.columns = pd.Series(df.columns).map(lambda x:dictionary[x])
        tmp = df
        if verbose==True:
            print("Successfully mapped columns for df.")
        dropidx =[None==col for col in list(tmp.columns)]
        tmp=tmp.drop(columns=df.columns[dropidx])
        if verbose==True:
            print("Successfully dropped unnecessary columns for df.")
    except:
        tmp = None
        print("Skipping mapndrop call for df.")
    return tmp


[Back to: Functions](#Functions)

In [8]:
def namefile(name, tzadjust=5,tzdirection = '-', adjprecision='minutes', filetype = 'csv'):
    """takes a filename and filetype, and adds a timestamp adjusted relative to gmt to a precision 
    and returns a string that concatenates them."""
    assert isinstance(name,str),"'name' must be of type str."
    assert isinstance(tzadjust,int),"'tzadjust' must be of type int"
    assert adjprecision in ['date','hour','minutes','seconds', 'max'], "'adjprecision' must be either \
    'date', hour','minutes','seconds', or 'max'"
    precision= {'max':None,'seconds':-7,'minutes':-9, 'hours':-14,'date':-20}
    if tzdirection== '-':
        timestamp = (pd.to_datetime('now')-pd.Timedelta(hours=tzadjust))
    else:
        timestamp = (pd.to_datetime('now')+pd.Timedelta(hours=int(tzadjust[1:])))
    timestamp = str(timestamp).replace(':','hrs',1).replace(':','min',1)
    timestamp = timestamp[:precision[adjprecision]]
    filename = name+'_' + timestamp+ '.' +filetype
    return filename


## Preparing for a Save
[Top](#TOC)

[Setting up Python](#SettingUp)

## Setting the Location
[Top](#TOC)

[Setting Up Python](#SettingUp)

These chunks identify the locations from which we can get data and to which we can save data.

### Source Data
Raw data can be found in the following locations:

In [9]:
# sourceDataPers = 'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
# sourceDataBig = 'S:/Chris/TailDemography/TailDemography/Raw Data'
# sourceBlack = 'C:/Users/test/Desktop'
sourceGandolf = 'C:/Users/craga/Google Drive/TailDemography/Raw Data'


### Intermediate Source Data
Intermediate files can be found in the following locations:

In [10]:
# sourceInterDataPers = 'C:/Users/Christopher/Google Drive/TailDemography/Intermediate Files/Source'
# sourceinterDataBig = 'S:/Chris/TailDemography/TailDemography/Intermediate Files/Source'
# sourceBlack = 'C:/Users/test/Desktop'
sourceInterGandolf = 'C:/Users/craga/Google Drive/TailDemography/Intermediate Files/Source'

Now we change the working directory to the source path.

In [11]:
os.chdir(sourceGandolf)

### Output Data
The cleaned data will be saved to one of these locations:

In [12]:
# outputPers = 'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
# outputBig = 'S:/Chris/TailDemography/TailDemography/Cleaned Combined Data'
# outputBlack = 'C:/Users/test/Desktop'
outputGandolf = 'C:/Users/craga/Google Drive/TailDemography/Cleaned Combined Data'

### Review files

In [13]:
# outputPers = 'C:/Users/Christopher/Google Drive/TailDemography/Files for review/Source files'
# reviewfolderBig = 'S:/Chris/TailDemography/TailDemography/Files for review/Source files'
reviewGandolf = 'C:/Users/craga/Google Drive/TailDemography/Files for review/Source files'

# Handling Columns
[Top](#TOC)

We don't have to look in the multiple-sheet file.  It's clear that we'll have to identify a common set of columns prior to combining these files.  Let's define a few functions to help us do this.

We will want to do the following:
1. [Find Unique Column Names](#FindUniqueCol)
2. [Eliminate Unnecessary Columns](#DropCol)
3. [Combine Synonyms](#CombineCol)

Here we use search the source path to locate and eventually read the raw data into our notebook.

In [14]:
rawfiles = glob.glob('*.xls*')
rawfiles

['CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19.xlsx',
 'CC 2004.xlsx',
 'CC 2015 - captures.xls',
 'CC 2016 - captures.xls',
 'CC 2017 Lizards - 3viii17 captures and obs.xls',
 'xCC2005x.xls',
 'xCC2006x.xls',
 'xCC2007x.xls',
 'xCC2008x.xls',
 'xCC2009x.xls',
 'xCC2010x.xlsx',
 'xCC2011x.xls',
 'xCC2012x.xls',
 'xCC2013x.xls',
 'xCC2014x.xlsx']

We'll separate these into files with single or multiple sheets.

In [15]:
rawfiles_ms = [rawfiles[0],rawfiles[7]]
rawfiles_ss = list(set(rawfiles)- set(rawfiles_ms))

The names of files with multiple sheets are now in the variable *rawfiles_ms*.

In [16]:
rawfiles_ms

['CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19.xlsx',
 'xCC2007x.xls']

The names of files with a single sheet are now in the variable *rawfiles_ss*.

In [17]:
rawfiles_ss

['xCC2013x.xls',
 'xCC2005x.xls',
 'xCC2008x.xls',
 'xCC2009x.xls',
 'xCC2011x.xls',
 'xCC2006x.xls',
 'xCC2010x.xlsx',
 'xCC2012x.xls',
 'CC 2015 - captures.xls',
 'CC 2004.xlsx',
 'CC 2017 Lizards - 3viii17 captures and obs.xls',
 'xCC2014x.xlsx',
 'CC 2016 - captures.xls']

Now let's take a look at the number of columns in each file. We'll start with the single sheet files, since this is the easiest.  We will use the function, *xlcolshape* to make this easier. 
When we call this function on the first of the single-sheet files, we can see that it returns a tuple in the format ('number of rows', 'number of columns'). The code for *xlcolshape* can be found in [Functions](#functions).

In [18]:
xlcolshape(rawfiles_ss[0])

Doing stuff you asked me to do for file 'xCC2013x.xls',sheet 'CC 2013 data' programmer person.


{'xCC2013x.xls_CC 2013 data': (106, 20)}

We will apply this function to the list of files for our inspection.

In [19]:
pd.Series(rawfiles).apply(lambda x: xlcolshape(x,verbose=False))

0     {'CC 2000-03-modified from CC-SJ 00-03 final-m...
1                     {'CC 2004.xlsx_2004 ': (479, 16)}
2            {'CC 2015 - captures.xls_2015': (241, 19)}
3            {'CC 2016 - captures.xls_2016': (103, 21)}
4     {'CC 2017 Lizards - 3viii17 captures and obs.x...
5                      {'xCC2005x.xls_2005': (202, 17)}
6                      {'xCC2006x.xls_2006': (163, 17)}
7     {'xCC2007x.xls_Sheet1': (507, 16), 'xCC2007x.x...
8                      {'xCC2008x.xls_2008': (134, 20)}
9                      {'xCC2009x.xls_2009': (162, 16)}
10                   {'xCC2010x.xlsx_Sheet1': (99, 41)}
11                    {'xCC2011x.xls_Sheet1': (64, 19)}
12                      {'xCC2012x.xls_data': (85, 19)}
13             {'xCC2013x.xls_CC 2013 data': (106, 20)}
14                     {'xCC2014x.xlsx_2014': (97, 19)}
dtype: object

## Finding Unique Columns
[Top](#TOC)

[Handling Columns](#HandlingColumns)

We'll use the function, *xluniqucol2* to extract column names and convert them to an approved set.  We'll use that function to allow us to only add unique names to a list of column names. 

Here is an example of how xluniquecol2 works for a file with one sheet.  You can find the code for *xluniquecol2* in [Functions](#functions).

In [20]:
xluniquecol2(rawfiles_ss[0],verbose=False)

['SVL',
 'TL',
 'Location',
 'Mark',
 'Painted',
 'Species',
 'Sex',
 'Mass',
 'Meters',
 'Unnamed: 17',
 'New/Recap',
 'Misc.',
 'Spotted',
 'Paint Mark',
 'Time',
 'Toes',
 'Date',
 'RTL',
 'Unnamed: 0',
 'Vial']

Here is an example of how xluniquecol2 works for a file with multiple sheets.

In [21]:
xluniquecol2(rawfiles_ms[0],verbose=False)

['SVL',
 'Unnamed: 16',
 'misc',
 'TL',
 'paint mark',
 'TOES',
 'TIME',
 'meters',
 'species',
 'location',
 'RTL',
 'date',
 'mass',
 'sex',
 'painted or not',
 'VIAL',
 'NEW/recap']

Now we will create an empty set, *uniquecols2*, that will eventually contain the unique column names in all of the files.

We will append the unique column names from each file to *uniquecols2*.

In [22]:
tmp = pd.Series(rawfiles).apply(xluniquecol2,verbose=False)
uniquecols2 = list()
for u in tmp:
    uniquecols2 = uniquecols2+u
uniquecols2 = list(set(uniquecols2))
uniquecols2

['SVL',
 1,
 'TL',
 'paint mark',
 'Location',
 '1st Capture (year)',
 'TOES',
 'Mark',
 'species',
 'Toe 1',
 'RTL (mm)',
 'Toe 6',
 'Painted',
 'Toe 11',
 'Species',
 'Collectors',
 'TL (mm)',
 'date',
 'mass',
 'Sex',
 'Mass',
 'SVL (mm)',
 'misc/notes',
 'Toe 5',
 'Meters',
 'Marked',
 'Toe 13',
 'Toe 20',
 'Year',
 'Unnamed: 17',
 'location',
 'painted',
 'Toe 9',
 'Toe 18',
 'Unnamed: 19',
 'New/Recap',
 'Toe 7',
 'NEW/recap',
 'Unnamed: 16',
 'mass (g)',
 'Toe 14',
 ' painted or not',
 'Toe 8',
 'TIME',
 'Toe 2',
 'Toe 4',
 'Misc.',
 'meters',
 'Spotted',
 'Toe 3',
 'Tail condition (1=intact; 2=autotomized; 3=regrown)',
 'Toe 19',
 'Years Alive (known)',
 'painted or not',
 'Paint Mark',
 'Time',
 'misc',
 'Toe 16',
 'Toe 17',
 'Toes',
 'Date',
 'Painted or Not',
 'Toe 10',
 'RTL',
 'Unnamed: 0',
 'Toe 15',
 'Toe 12',
 'Vial',
 'sex',
 '2015 or earlier',
 'VIAL']

## Eliminate Unnecessary Columns
[Top](#TOC)

[Cleaning Data](#CleaningData)

[Handling Columns](#HandlingColumns)

Now we will try to identify unnecessary columns and eliminate them. Much of this will be done manually.

In [23]:
keepCol = ['species', 'date', 'sex', 'svl', 'tl', 'rtl', 'mass',
       'paint.mark', 'location', 'meters', 'new.recap', 'painted', 'misc',
       'vial', 'autotomized', 'sighting', 'toes','filename']

In [24]:
set(pd.Series(keepCol).str.lower())-set(pd.Series(uniquecols2).str.lower())

{'autotomized', 'filename', 'new.recap', 'paint.mark', 'sighting'}

In [25]:
set(pd.Series(uniquecols2).str.lower())-set(pd.Series(keepCol).str.lower())

{' painted or not',
 '1st capture (year)',
 '2015 or earlier',
 'collectors',
 'mark',
 'marked',
 'mass (g)',
 'misc.',
 'misc/notes',
 nan,
 'new/recap',
 'paint mark',
 'painted or not',
 'rtl (mm)',
 'spotted',
 'svl (mm)',
 'tail condition (1=intact; 2=autotomized; 3=regrown)',
 'time',
 'tl (mm)',
 'toe 1',
 'toe 10',
 'toe 11',
 'toe 12',
 'toe 13',
 'toe 14',
 'toe 15',
 'toe 16',
 'toe 17',
 'toe 18',
 'toe 19',
 'toe 2',
 'toe 20',
 'toe 3',
 'toe 4',
 'toe 5',
 'toe 6',
 'toe 7',
 'toe 8',
 'toe 9',
 'unnamed: 0',
 'unnamed: 16',
 'unnamed: 17',
 'unnamed: 19',
 'year',
 'years alive (known)'}

Since data for years 2000-2003 are contained in the same Excel file we will have to treat this file differently than the others.

## Combining Synonymous Columns
[Top](#TOC)

[Cleaning Data](#CleaningData)

[Handling Columns](#HandlingColumns)

Once we have identified the columns we need to keep, we'll need to apply this list to the files as they are read into python by doing the following:

We will use a function, *colmatchtodict*,  to identify potential synonyms. Here's an example of how *colmatchtodict* works.  The code for this function can be found in [Functions](#functions).

In [26]:
coldict = {}

In [27]:
colmatchtodict('toes',pd.Series(uniquecols2),coldict, key = 'toes')

{'toes': ['TOES', 'Toes']}

Now let's see what happened when we apply this funtion to our, keepCol.

In [28]:
coldict = {}

In [29]:
pd.Series(keepCol).apply(lambda x: colmatchtodict(x=x,series=pd.Series(uniquecols2),dictsource=coldict))
coldict

{'species': ['species', 'Species'],
 'date': ['date', 'Date'],
 'sex': ['Sex', 'sex'],
 'svl': ['SVL', 'SVL (mm)'],
 'tl': ['TL', 'RTL (mm)', 'TL (mm)', 'RTL'],
 'rtl': ['RTL (mm)', 'RTL'],
 'mass': ['mass', 'Mass', 'mass (g)'],
 'paint.mark': ['paint mark', 'Paint Mark'],
 'location': ['Location', 'location'],
 'meters': ['Meters', 'meters'],
 'new.recap': ['New/Recap', 'NEW/recap'],
 'painted': ['Painted',
  'painted',
  ' painted or not',
  'painted or not',
  'Painted or Not'],
 'misc': ['misc/notes', 'Misc.', 'misc'],
 'vial': ['Vial', 'VIAL'],
 'autotomized': ['Tail condition (1=intact; 2=autotomized; 3=regrown)'],
 'sighting': [],
 'toes': ['TOES', 'Toes'],
 'filename': []}

We will manually adjust the values for 'tl' and 'filename'.

In [30]:
coldict['tl']=['TL (mm)', 'TL', 'tl']

Now we need to use this dict to relabel the columns we wish to keep.

We will use the function, *findsyn* to identify potential synonymous to the columnlabels in *keepcols* among the column labels in *uniquecols2*. 

Here is are a few examples of how *findsyn* works.  The code can be found in [Functions](#functions).

In [31]:
findsyn('RTi',coldict,verbose=False)

In [32]:
findsyn('RTi',coldict,verbose=True)

No value matching "RTi" was found in the dictionary.


In [33]:
findsyn('RTL',coldict,verbose=True)

'rtl'

Now we apply *findsyn* to *uniquecol* and create a column of synonyms.

In [34]:
uniquecols2df = pd.DataFrame({'uniquecols2':uniquecols2})
uniquecols2df['preferredcol'] = uniquecols2df.uniquecols2.apply(lambda x: findsyn(x,coldict,False))
uniquecols2df

,uniquecols2,preferredcol
0,SVL,svl
1,1,None
2,TL,tl
3,paint mark,paint.mark
4,Location,location
5,1st Capture (year),None
6,TOES,toes
7,Mark,None
8,species,species
9,Toe 1,None


Now we will turn this dataframe back into a dict so that we can easily use it to rename columns

In [35]:
uniquecols2df.index = uniquecols2df.uniquecols2
uniquecols2dict = pd.Series(uniquecols2df.preferredcol).to_dict()
uniquecols2dict

{'SVL': 'svl',
 1: None,
 'TL': 'tl',
 'paint mark': 'paint.mark',
 'Location': 'location',
 '1st Capture (year)': None,
 'TOES': 'toes',
 'Mark': None,
 'species': 'species',
 'Toe 1': None,
 'RTL (mm)': 'rtl',
 'Toe 6': None,
 'Painted': 'painted',
 'Toe 11': None,
 'Species': 'species',
 'Collectors': None,
 'TL (mm)': 'tl',
 'date': 'date',
 'mass': 'mass',
 'Sex': 'sex',
 'Mass': 'mass',
 'SVL (mm)': 'svl',
 'misc/notes': 'misc',
 'Toe 5': None,
 'Meters': 'meters',
 'Marked': None,
 'Toe 13': None,
 'Toe 20': None,
 'Year': None,
 'Unnamed: 17': None,
 'location': 'location',
 'painted': 'painted',
 'Toe 9': None,
 'Toe 18': None,
 'Unnamed: 19': None,
 'New/Recap': 'new.recap',
 'Toe 7': None,
 'NEW/recap': 'new.recap',
 'Unnamed: 16': None,
 'mass (g)': 'mass',
 'Toe 14': None,
 ' painted or not': 'painted',
 'Toe 8': None,
 'TIME': None,
 'Toe 2': None,
 'Toe 4': None,
 'Misc.': 'misc',
 'meters': 'meters',
 'Spotted': None,
 'Toe 3': None,
 'Tail condition (1=intact; 2=autoto

We'll use the dict, *uniquecols2dict* to rename the synonymous columns in our file....once we read them in,
that is.

# Reading and Appending Data
[Top](#TOC)

Now we use the function *readnsplit* to actually read in the source files, drop unnecessary columns and renaming columns according to a dictionary. 

Here is an example of how *readnsplit* works.  The code can be found in [Functions](#functions).

In [37]:
readnsplit(rawfiles[0],sourceInterGandolf,str)

Success!  'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19.xlsx',sheet '2000' has been saved to C:/Users/craga/Google Drive/TailDemography/Intermediate Files/Source and the corresponding                google drive file as C:/Users/craga/Google Drive/TailDemography/Intermediate Files/Source/CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2000.xlsx.
Success!  'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19.xlsx',sheet '2001' has been saved to C:/Users/craga/Google Drive/TailDemography/Intermediate Files/Source and the corresponding                google drive file as C:/Users/craga/Google Drive/TailDemography/Intermediate Files/Source/CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2001.xlsx.
Success!  'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19.xlsx',sheet '2002' has been saved to C:/Users/craga/Google Drive/TailDemography/Intermediate Files/Source and the corresponding       

In [38]:
for file in rawfiles:
    readnsplit(file,sourceInterGandolf,dtype=str, verbose=False)

We need to change the directory to the location where the intermediate files this operates on can be found.  We will also save a list of the files names in that location for convenience.

In [39]:
os.chdir(sourceInterGandolf)
splitfiles = glob.glob('*xls*')
splitfiles

['CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2000.xlsx',
 'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2001.xlsx',
 'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2002.xlsx',
 'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2003.xlsx',
 'CC 2004_2004 .xlsx',
 'CC 2015 - captures_2015.xls',
 'CC 2016 - captures_2016.xls',
 'CC 2017 Lizards - 3viii17 captures and obs_2017.xls',
 'xCC2005x_2005.xls',
 'xCC2006x_2006.xls',
 'xCC2007x_2007.xls',
 'xCC2007x_Sheet1.xls',
 'xCC2008x_2008.xls',
 'xCC2009x_2009.xls',
 'xCC2010x_Sheet1.xlsx',
 'xCC2011x_Sheet1.xls',
 'xCC2012x_data.xls',
 'xCC2013x_CC 2013 data.xls',
 'xCC2014x_2014.xlsx']

Now we remove 'xCC2007x_Sheet1.xls' from the list of files we will process intermediate files since this is a subset of the 'xCC2007x_2007.xls' reordered and with some columns dropped.

In [40]:
splitfiles = list(set(splitfiles)-set(['xCC2007x_Sheet1.xls']))
splitfiles

['CC 2015 - captures_2015.xls',
 'xCC2007x_2007.xls',
 'xCC2014x_2014.xlsx',
 'CC 2004_2004 .xlsx',
 'xCC2006x_2006.xls',
 'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2002.xlsx',
 'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2003.xlsx',
 'xCC2013x_CC 2013 data.xls',
 'xCC2009x_2009.xls',
 'xCC2012x_data.xls',
 'xCC2010x_Sheet1.xlsx',
 'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2001.xlsx',
 'CC 2017 Lizards - 3viii17 captures and obs_2017.xls',
 'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2000.xlsx',
 'xCC2011x_Sheet1.xls',
 'CC 2016 - captures_2016.xls',
 'xCC2005x_2005.xls',
 'xCC2008x_2008.xls']

Now we use the function *mapndrop* to drop unnecessary columns and renaming columns according to a dictionary.

Here are a few examples of how *mapndrop* works.  The code can be found in [Functions](#functions).

In [41]:
mapndrop(df=pd.read_excel(splitfiles[0],dtype=str),dictionary=uniquecols2dict,verbose = True)

Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.


,species,toes,date,sex,svl,tl,rtl,mass,paint.mark,location,meters,new.recap,painted,misc,vial
0,j,5-7-13,2015-07-06 00:00:00,f,67,97,0,9.7,w2a,L 5m^ bottom site,-35,new,yes,B recently shed; TSS,15-02
1,j,5-7-14-20,2015-07-06 00:00:00,m,73,71,33,14.3,w21a,SB 5m,-25,new,yes,NaN,15-16
2,j,5-7-13-16,2015-07-06 00:00:00,m,78,75,55,14.8,w3a,SB 8m v bowl,-16,new,yes,NaN,15-03
3,j,5-11,2015-07-06 00:00:00,f,75,50,15,11.3,w6a,L SB^ 1 falls,1,new,yes,NaN,NaN
4,j,5-7-13-17,2015-07-06 00:00:00,f,66,84,0,10,w7a.c,SB 20^1falls,20,new,yes,shedding; poop 15-101,15-04
5,j,5-7-14-19,2015-07-06 00:00:00,m,78,94,24,16,w20a,bottom curved wall v cave trail,35,new,yes,NaN,15-15
6,j,5-15-16,2015-07-06 00:00:00,f,70,93,0,11.3,w8a.c,top L rock [stacked] wall ^ cave trail,80,new,yes,BSS; T shed,NaN
7,j,5-7-13-18,2015-07-06 00:00:00,f,66,88,0,9.2,w9a,bottom rock wall^ RW at juniper xing,83,new,yes,NaN,15-05
8,j,5-7-13-19,2015-07-06 00:00:00,f,72,97,0,12.8,w10a.t,SB5m below root xing,153,new,yes,[lost 2 of the cut toes],15-75
9,j,5-7-13-20,2015-07-06 00:00:00,m,73,76,4,12.5,w11,rock wall -> black rock,171,new,yes,BSS; TSS; orange color scattered throughout bo...,NaN


In [42]:
mapndrop(df=pd.read_excel(splitfiles[4],dtype=str),dictionary=uniquecols2dict, verbose=True)

Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.


,species,toes,date,sex,svl,tl,rtl,mass,paint.mark,location,meters,new.recap,painted,misc,vial
0,sj,NaN,2006-05-21 00:00:00,NaN,NaN,NaN,NaN,NaN,w4c,1 falls,0,Dead,NaN,Dead on ground !?,NaN
1,sc,NaN,2006-05-31 00:00:00,NaN,NaN,NaN,NaN,NaN,???,juniper left side 8m v top CCC,240,heard,NaN,NaN,NaN
2,as,15,2006-05-28 00:00:00,NaN,83,185,0,17,w.bc,opp H4.5,200,NEW,painted,66-06,NaN
3,as,16,2006-05-28 00:00:00,NaN,57,159,0,7.8,w.c,L@ CC/CCC,240,NEW,painted,65-06,NaN
4,sc,19,2006-06-01 00:00:00,m,96,130,0,29,w.at,right juniper at CC/CCC,237,NEW,painted,77-06,NaN
5,sc,20,2006-05-21 00:00:00,m,106,129,0,43.4,w.a,downed oak 5m ^ 2 triple R rt side [below not ...,330,NEW,painted,pictures taken,28-06
6,sj,4 6 14 16,2006-05-20 00:00:00,m,62,78,0,7.8,w1t,left sb 5m v 1falls,-5,NEW,painted,Tshed;Bss,01-06
7,sj,4 6 14 18,2006-05-20 00:00:00,f,58,74,0,7.6,w2b,left sb 5m v 1falls,-5,NEW,painted,NaN,02-06
8,sj,4 6 14 19,2006-05-20 00:00:00,f,52,76,0,4.3,w5b,4m v 1 falls left side,-4,NEW,painted,NaN,03-06
9,sj,4 6 14 20,2006-05-20 00:00:00,f,62,86,0,8,w6b.c,sb wall left 20m ^ 1 falls,20,NEW,painted,NaN,04-06


We'll create a df, *df*, with no data, but columns from our desired columns, *i.e.* the keys for coldict, as a placeholder to which we can append new data.

In [43]:
df = pd.DataFrame(columns=coldict.keys())
df

,species,date,sex,svl,tl,rtl,mass,paint.mark,location,meters,new.recap,painted,misc,vial,autotomized,sighting,toes,filename


Now we will read in all of the successfully split files, clean the column names, and concatenate them into one large df.

In [44]:
for file in splitfiles:
    df = pd.concat([df,mapndrop(pd.read_excel(file,dtype=str),uniquecols2dict)],sort=True)
    print(df.shape[0])
print("\n\nFinal df has {} columns and {} rows.".format(df.shape[1],df.shape[0]))
df.head()

Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
241
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
423
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
520
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
999
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
1162
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
2639
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
3656
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
3762
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
3924
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
4009
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
4108
Successfully m

,autotomized,date,filename,location,mass,meters,misc,new.recap,paint.mark,painted,rtl,sex,sighting,species,svl,tl,toes,vial
0,NaN,2015-07-06 00:00:00,NaN,L 5m^ bottom site,9.7,-35,B recently shed; TSS,new,w2a,yes,0,f,NaN,j,67,97,5-7-13,15-02
1,NaN,2015-07-06 00:00:00,NaN,SB 5m,14.3,-25,NaN,new,w21a,yes,33,m,NaN,j,73,71,5-7-14-20,15-16
2,NaN,2015-07-06 00:00:00,NaN,SB 8m v bowl,14.8,-16,NaN,new,w3a,yes,55,m,NaN,j,78,75,5-7-13-16,15-03
3,NaN,2015-07-06 00:00:00,NaN,L SB^ 1 falls,11.3,1,NaN,new,w6a,yes,15,f,NaN,j,75,50,5-11,NaN
4,NaN,2015-07-06 00:00:00,NaN,SB 20^1falls,10,20,shedding; poop 15-101,new,w7a.c,yes,0,f,NaN,j,66,84,5-7-13-17,15-04


In [45]:
df = df.reindex(['species', 'toes', 'sex', 'date', 'svl', 'tl', 'rtl', 'autotomized', 'mass', 
                 'location', 'meters', 'new.recap', 'painted', 'sighting', 
                 'paint.mark', 'vial', 'misc'], axis=1)
df.head()

,species,toes,sex,date,svl,tl,rtl,autotomized,mass,location,meters,new.recap,painted,sighting,paint.mark,vial,misc
0,j,5-7-13,f,2015-07-06 00:00:00,67,97,0,NaN,9.7,L 5m^ bottom site,-35,new,yes,NaN,w2a,15-02,B recently shed; TSS
1,j,5-7-14-20,m,2015-07-06 00:00:00,73,71,33,NaN,14.3,SB 5m,-25,new,yes,NaN,w21a,15-16,NaN
2,j,5-7-13-16,m,2015-07-06 00:00:00,78,75,55,NaN,14.8,SB 8m v bowl,-16,new,yes,NaN,w3a,15-03,NaN
3,j,5-11,f,2015-07-06 00:00:00,75,50,15,NaN,11.3,L SB^ 1 falls,1,new,yes,NaN,w6a,NaN,NaN
4,j,5-7-13-17,f,2015-07-06 00:00:00,66,84,0,NaN,10,SB 20^1falls,20,new,yes,NaN,w7a.c,15-04,shedding; poop 15-101


In [46]:
df.shape

(6299, 17)

# Exporting Data
[Top](#TOC)

Here we call the function, *namefile*, to create a timestamped name for file to be exported.  You can find the code for *namefile* in [Functions](#functions).

In [47]:
filename = namefile('Appended and Trimmed CC Data 2000-2017')
os.chdir(outputGandolf)
df.to_csv(filename,index = False)
print("\'{}\' has been saved to \'{}\' and the corresponding drive google drive location."\
      .format(filename, outputGandolf))

'Appended and Trimmed CC Data 2000-2017_2020-02-15 18hrs53min.csv' has been saved to 'C:/Users/craga/Google Drive/TailDemography/Cleaned Combined Data' and the corresponding drive google drive location.


# Where to Next
[Back to Top](#Table-of-Contents)

Next proceed to Cleaning CC (Part 2).